# 🧪 RAG con LLM ligero usando arquitectura del proyecto
Este notebook usa los módulos definidos en la carpeta `src/` y `create_database.py` desde la raíz del proyecto.

Se realiza recuperación basada en embeddings y generación con LLM `gemma3:1b` vía Ollama.

In [ ]:
# Configuración de entorno para imports relativos
import sys
from pathlib import Path

# Añadir raíz del proyecto y src al path
BASE_DIR = Path("..").resolve()
SRC_DIR = BASE_DIR / "src"
sys.path.append(str(BASE_DIR))
sys.path.append(str(SRC_DIR))

from data_processing import convert_to_md_using_docling
from graph_db import GraphDB
from chatbot import Chatbot
import create_database  # Este se usará si hay procesamiento de chunks

from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from datetime import datetime
import json

In [ ]:
# Parámetros de prueba
query = "¿Por qué es importante el análisis matemático en ciencia de datos?"
n_docs = 3

# Documentos de prueba localmente embebidos
raw_docs = [
    "El análisis matemático proporciona herramientas fundamentales para el modelado y la optimización de algoritmos en ciencia de datos.",
    "Sin cálculo diferencial e integral, sería imposible entender cómo cambian los valores en una red neuronal.",
    "Los fundamentos de álgebra lineal son esenciales para el manejo de vectores, matrices y transformaciones en espacios de alta dimensión.",
    "Estadística y probabilidad también se consideran parte del análisis matemático aplicado, especialmente en modelos predictivos."
]

In [ ]:
# Indexar documentos usando OllamaEmbeddings y Chroma
embedding = OllamaEmbeddings(model="gemma:2b")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
docs = [Document(page_content=chunk) for doc in raw_docs for chunk in text_splitter.split_text(doc)]
vectorstore = Chroma.from_documents(documents=docs, embedding=embedding, persist_directory=".chroma_index")

In [ ]:
# Recuperación de documentos relevantes
retrieved_docs = vectorstore.similarity_search(query, k=n_docs)
contexto = "\n\n".join([doc.page_content for doc in retrieved_docs])
print("📄 Documentos recuperados:\n")
for i, doc in enumerate(retrieved_docs):
    print(f"[{i}] {doc.page_content}\n")

In [ ]:
# Generar respuesta usando Chatbot definido en src/chatbot.py
chatbot = Chatbot(
    base_prompt="Eres un tutor académico que responde en español basándote en contexto recuperado."
)

prompt = f"""
Contexto:
{contexto}

Pregunta:
{query}
"""

respuesta = chatbot(prompt)
print("\n📘 Respuesta generada:\n")
print(respuesta)

In [ ]:
# Guardar resultado
resultado = {
    "query": query,
    "contexto": contexto,
    "respuesta": respuesta,
    "timestamp": datetime.now().isoformat()
}

filename = f"resultados_rag_integration_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(filename, "w", encoding="utf-8") as f:
    json.dump(resultado, f, indent=2, ensure_ascii=False)

print(f"✅ Resultado guardado en {filename}")